In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 199 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ac9b6504f020c70983e4a0332caf11e93e47cc43365ad670e68f4fc77a6036e6
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer, IDF
from pyspark.ml.classification import  LogisticRegression
from pyspark.sql import SparkSession

if __name__ == "__main__":
  spark = SparkSession \
          .builder \
          .appName("pipeline_ex") \
          .getOrCreate()

new_data = spark.createDataFrame([
    (0, "3 5 6 ayse fatma nil", 1.0),
    (1, "nursel zuleyha gamze 8 9", 0.0 ),
    (2, "kutuphane spark nil 5", 1.0),
    (3, "kitaplar 56 bilgisayar", 0.0),
    (4, "nil cay mentor 9", 1.0),
    (5, "2 agaclar yildiz", 0.0)
], ["id", "text", "label"])

tokenizer = Tokenizer(inputCol= "text", outputCol="words")
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="features")
log_reg = LogisticRegression(maxIter= 10, regParam= 0.001)
pipeline = Pipeline(stages = [tokenizer, hashing_tf, log_reg])

model = pipeline.fit(new_data)

test_data = spark.createDataFrame([
    (6, "kalem 4 su apache"),
    (7, "ai text nil"),
    (8, "kurs cassandra university"),
    (9, "9 8 nil 7")
], ["id", "text"])

prediction = model.transform(test_data)
selected = prediction.select("id", "text", "probability", "prediction")
for r in selected.collect():
    row_id, text, probability, prediction = r
    print("{}, {} prob= {}, pred={} ".format(row_id, text, probability, prediction))

spark.stop()

6, kalem 4 su apache prob= [0.7869286716251876,0.21307132837481235], pred=0.0 
7, ai text nil prob= [0.2035313652070624,0.7964686347929376], pred=1.0 
8, kurs cassandra university prob= [0.7869286716251876,0.21307132837481235], pred=0.0 
9, 9 8 nil 7 prob= [0.4028901338684307,0.5971098661315692], pred=1.0 
